In [1]:
from pymongo import MongoClient
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, element_at, when
import pygeohash as pgh 
import pandas as pd 
import json 
from datetime import datetime

In [2]:
topic = 'PartB'
client = MongoClient()
mydb = client.fit3182_assignment_db
streaming_climate_collection = mydb['streaming_climate_collection']
streaming_hotspot_collection = mydb['streaming_hotspot_collection']
streaming_hotspot_collection.drop()
streaming_climate_collection.drop()

Initialize our spark session with `#threads = #logicalCPU` and the given application name.

In [3]:
spark = (
    SparkSession.builder
    .master('local[*]')
    .appName('[Demo] Spark Streaming from Kafka into MongoDB')
    .getOrCreate()
)

Create a streaming dataframe with options providing the bootstrap server(s) and topic name.

In [4]:
topic_stream_df = (
    spark.readStream.format('kafka')
    .option('kafka.bootstrap.servers', 'localhost:9092')
    .option('subscribe', topic)
    .load()
)

In [5]:
topic_stream_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [6]:
output_stream_df = (
    topic_stream_df
    .select(                                     
            topic_stream_df.value.cast('string')
    .alias('data'))
)

In [7]:
output_stream_df.printSchema()

root
 |-- data: string (nullable = true)



In [8]:
class DbWriter:
    # called at the start of processing each partition in each output micro-batch
    def open(self, partition_id, epoch_id):
        self.mongo_client = MongoClient(
            host='localhost',
            port=27017
        )
        self.db = self.mongo_client['fit3182_db']
        return True
    
    # called once per row of the result dataframe
    # the current code DOES NOT handle duplicate processing
    #   e.g., query fails and restarts just before current micro-batch was fully inserted
    def process(self, row):
        self.db[topic_name].insert_one(row.asDict())
    
    # called once all rows have been processed (possibly with error)
    def close(self, err):
        self.mongo_client.close()

In [9]:
def process_batch(batch_df,batch_id):
    #raw_data = batch_df.collect()
    # raw _data = [Row(),Row()]
    # process data into [{},{},...]
    data = [row.asDict(row) for row in batch_df.collect()]
    lst = [] 
    climate_lst = []
    hotspot_lst = [] 
    for i in range(len(data)): 
        temp = data[i]['data']
        temp = eval(temp) 
        if temp['Producer_id'] == 'Producer_01': 
            climate_lst.append(temp)

        else: 
            hotspot_lst.append(temp)    
    
    # 1. check if there is climate data in this array 
    # no climate, return
    
    if len(climate_lst) == 0: 
        return 
    
    for i in climate_lst:
        date = i['Created_Day'][0:10]

#     if len(climate_lst) == 0: 
#         return 
    # create datetime str and datetime datetime format for further purpose to group in visualization
    for i in range(len(hotspot_lst)): 
        hotspot_lst[i]['Date'] = date 
        x = hotspot_lst[i]['Created_Date']
        Create_datetime = date + " " + x 
        Create_datetime_str = Create_datetime
        hotspot_lst[i]['Create_datetime_str'] = Create_datetime_str
        Create_datetime = datetime.strptime(Create_datetime,"%Y-%m-%d %H:%M:%S")
        hotspot_lst[i]['Created_Date'] = Create_datetime
    
   
    

    # 2. compute geohash for climate, hotspot(TERRA & AQUA) (done in producer)
    if len(climate_lst) > 0:
        for i in range(len(climate_lst)): 
            climate_lst[i]["Geohash3"] = pgh.encode(climate_lst[i]['latitude'],climate_lst[i]['longitude'],precision=3)
    
    for i in range(len(hotspot_lst)): 
        hotspot_lst[i]["Geohash3"] = pgh.encode(hotspot_lst[i]['latitude'],hotspot_lst[i]['longitude'],precision=3)
        hotspot_lst[i]["Geohash5"] = pgh.encode(hotspot_lst[i]['latitude'],hotspot_lst[i]['longitude'],precision=5)
     
    # 3. within same batch, group the climate and hotspot in a lst (==geohash_3)
    same_geohash_3 = [] 
    #same_location.append(climate_lst)
    for i in hotspot_lst: 
        if climate_lst[0].get("Geohash3") == i.get('Geohash3'): 
            same_geohash_3.append(i)
    
    
    # 4. if no matching, just insert climate data into mongodb and return 
    if len(same_geohash_3) == 0:
        for i in climate_lst:
            b = {
            'station': 948703, 
            'date': i.get('Created_Day'),
            'air_temperature_celcius': i.get('air_temperature_celcius'),
            'relative_humidity': i.get('relative_humidity'),
            'windspeed_knots':i.get('windspeed_knots'),
            'max_wind_speed': i.get('max_wind_speed'),
            'precitipation': i.get('precitipation '), 
            'GHI_w/m2': i.get('GHI_w/m2'),
            'geohash3': i.get('Geohash3')
            
            }
            streaming_climate_collection.insert_one(b)
        return 

    
    # 5. if matching, groupby the same location(geohash5) and agg the avg
    hotspot_fire_lst = []
    unique_geohash5 = []
    surface_temp_sum = 0 
    confidence_sum = 0
    count = 1 
    for i in range(len(same_geohash_3)): 
        # check whether we agg on this geohash code
        if same_geohash_3[i]['Geohash5'] not in unique_geohash5:
            surface_temp_sum += same_geohash_3[i]['surface_temperature_celcius'] 
            confidence_sum += same_geohash_3[i]['confidence'] 
            for j in range(len(same_geohash_3)): 
                # skip checking itself
                if i != j :
                    if same_geohash_3[i]['Geohash5'] == same_geohash_3[j]["Geohash5"]: 
                        # count as in how many matches geohash5
                        count += 1 
                        surface_temp_sum += same_geohash_3[j]['surface_temperature_celcius'] 
                        confidence_sum += same_geohash_3[j]['confidence']
                        unique_geohash5.append(same_geohash_3[j]['Geohash5'])
        surface_temp_avg = surface_temp_sum/count 
        confidence_avg = confidence_sum/count 
        surface_temp_sum = 0 
        confidence_sum = 0 
        # only insert if > 0.0 to avoid some duplicate inserting
        if confidence_avg > 0.0:
            y = {'longitude':same_geohash_3[i]['longitude'],
                 'latitude': same_geohash_3[i]['latitude'], 
                  'confidence': confidence_avg, 
                  'surface_temperature':surface_temp_avg, 
                  'Date': same_geohash_3[i]['Created_Date'], 
                  'Date_str': same_geohash_3[i]['Date']
                 } 
            hotspot_fire_lst.append(y)   
        # reset count 
        count = 1
    
    
    # 6. insert data into mongodb 
    
    for i in climate_lst:
        a = {
            'station': 948703, 
            'date': i.get('Created_Day'),
            'air_temperature_celcius': i.get('air_temperature_celcius'),
            'relative_humidity': i.get('relative_humidity'),
            'windspeed_knots':i.get('windspeed_knots'),
            'max_wind_speed': i.get('max_wind_speed'),
            'precitipation': i.get('precipitation '), 
            'GHI_w/m2': i.get('GHI_w/m2'), 
            'geohash3': i.get('Geohash3')
        }
        climate_lst[0] = a 
        
    if len(hotspot_fire_lst) > 0 : 
        for i in climate_lst: 
            if i.get('air_temperature_celcius') > 20 and i.get('GHI_w/m2') > 180: 
                i['Fire_Cause'] = 'natural'
            else: 
                i['Fire_Cause'] = 'other'     
    
    # insert data into mongodb 
    
    for i in climate_lst:
        streaming_climate_collection.insert_one(i)
    
    for i in hotspot_fire_lst: 
        streaming_hotspot_collection.insert_one(i)
        
    
    
         

    print("Climate_lst: ",climate_lst) 
    print("Hotspot_lst: ",hotspot_lst)
    print("Same: ", same_geohash_3)
    print('Same Geohash:' , hotspot_fire_lst)
  

In [10]:
db_writer = (
    output_stream_df
    .writeStream
    .outputMode('append')
    .trigger(processingTime='10 seconds')
    .foreachBatch(process_batch)
)

In [11]:
db_writer.start()

Climate_lst:  [{'station': 948703, 'date': '2022-01-11', 'air_temperature_celcius': 14, 'relative_humidity': 48.2, 'windspeed_knots': 12.5, 'max_wind_speed': 19.0, 'precitipation': None, 'GHI_w/m2': 122, 'geohash3': 'r1m', 'Fire_Cause': 'other', '_id': ObjectId('628a70c025967c93f524f0ed')}]
Hotspot_lst:  [{'latitude': -37.7619, 'longitude': 142.8294, 'confidence': 71, 'surface_temperature_celcius': 46, 'Producer_id': 'TERRA_Producer_03', 'Created_Date': datetime.datetime(2022, 1, 11, 19, 12), 'Date': '2022-01-11', 'Create_datetime_str': '2022-01-11 19:12:00', 'Geohash3': 'r1m', 'Geohash5': 'r1m94'}, {'latitude': -37.562, 'longitude': 148.05, 'confidence': 61, 'surface_temperature_celcius': 72, 'Producer_id': 'AQUA_Producer_02', 'Created_Date': datetime.datetime(2022, 1, 11, 0, 0), 'Date': '2022-01-11', 'Create_datetime_str': '2022-01-11 00:00:00', 'Geohash3': 'r33', 'Geohash5': 'r3362'}, {'latitude': -35.541, 'longitude': 143.311, 'confidence': 82, 'surface_temperature_celcius': 63, 'P

Climate_lst:  [{'station': 948703, 'date': '2022-01-16', 'air_temperature_celcius': 15, 'relative_humidity': 48.1, 'windspeed_knots': 7.9, 'max_wind_speed': 12.0, 'precitipation': None, 'GHI_w/m2': 131, 'geohash3': 'r1s', 'Fire_Cause': 'other', '_id': ObjectId('628a70f225967c93f524f0f6')}]
Hotspot_lst:  [{'latitude': -36.4489, 'longitude': 144.7685, 'confidence': 93, 'surface_temperature_celcius': 73, 'Producer_id': 'TERRA_Producer_03', 'Created_Date': datetime.datetime(2022, 1, 16, 14, 24), 'Date': '2022-01-16', 'Create_datetime_str': '2022-01-16 14:24:00', 'Geohash3': 'r1w', 'Geohash5': 'r1wbw'}, {'latitude': -36.7034, 'longitude': 141.6168, 'confidence': 71, 'surface_temperature_celcius': 46, 'Producer_id': 'AQUA_Producer_02', 'Created_Date': datetime.datetime(2022, 1, 16, 19, 12), 'Date': '2022-01-16', 'Create_datetime_str': '2022-01-16 19:12:00', 'Geohash3': 'r1k', 'Geohash5': 'r1kxn'}, {'latitude': -38.141, 'longitude': 143.183, 'confidence': 68, 'surface_temperature_celcius': 46

Climate_lst:  [{'station': 948703, 'date': '2022-01-23', 'air_temperature_celcius': 23, 'relative_humidity': 60.6, 'windspeed_knots': 10.1, 'max_wind_speed': 26.0, 'precitipation': None, 'GHI_w/m2': 180, 'geohash3': 'r1m', 'Fire_Cause': 'other', '_id': ObjectId('628a713825967c93f524f101')}]
Hotspot_lst:  [{'latitude': -37.6857, 'longitude': 142.8532, 'confidence': 59, 'surface_temperature_celcius': 40, 'Producer_id': 'TERRA_Producer_03', 'Created_Date': datetime.datetime(2022, 1, 23, 14, 24), 'Date': '2022-01-23', 'Create_datetime_str': '2022-01-23 14:24:00', 'Geohash3': 'r1m', 'Geohash5': 'r1m9d'}, {'latitude': -36.1247, 'longitude': 145.1437, 'confidence': 71, 'surface_temperature_celcius': 46, 'Producer_id': 'AQUA_Producer_02', 'Created_Date': datetime.datetime(2022, 1, 23, 19, 12), 'Date': '2022-01-23', 'Create_datetime_str': '2022-01-23 19:12:00', 'Geohash3': 'r1x', 'Geohash5': 'r1x4q'}, {'latitude': -37.45, 'longitude': 148.097, 'confidence': 70, 'surface_temperature_celcius': 37

Climate_lst:  [{'station': 948703, 'date': '2022-01-27', 'air_temperature_celcius': 15, 'relative_humidity': 56.1, 'windspeed_knots': 5.1, 'max_wind_speed': 9.9, 'precitipation': None, 'GHI_w/m2': 122, 'geohash3': 'r1w', 'Fire_Cause': 'other', '_id': ObjectId('628a716025967c93f524f10a')}]
Hotspot_lst:  [{'latitude': -37.5052, 'longitude': 142.8636, 'confidence': 62, 'surface_temperature_celcius': 54, 'Producer_id': 'TERRA_Producer_03', 'Created_Date': datetime.datetime(2022, 1, 27, 14, 24), 'Date': '2022-01-27', 'Create_datetime_str': '2022-01-27 14:24:00', 'Geohash3': 'r1m', 'Geohash5': 'r1mdd'}, {'latitude': -36.5399, 'longitude': 144.678, 'confidence': 90, 'surface_temperature_celcius': 66, 'Producer_id': 'AQUA_Producer_02', 'Created_Date': datetime.datetime(2022, 1, 27, 19, 12), 'Date': '2022-01-27', 'Create_datetime_str': '2022-01-27 19:12:00', 'Geohash3': 'r1w', 'Geohash5': 'r1wbh'}, {'latitude': -37.9004, 'longitude': 141.0921, 'confidence': 68, 'surface_temperature_celcius': 44

Climate_lst:  [{'station': 948703, 'date': '2022-02-02', 'air_temperature_celcius': 14, 'relative_humidity': 52.8, 'windspeed_knots': 9.3, 'max_wind_speed': 18.1, 'precitipation': None, 'GHI_w/m2': 117, 'geohash3': 'r1k', 'Fire_Cause': 'other', '_id': ObjectId('628a719c25967c93f524f116')}]
Hotspot_lst:  [{'latitude': -37.394, 'longitude': 143.5541, 'confidence': 83, 'surface_temperature_celcius': 64, 'Producer_id': 'TERRA_Producer_03', 'Created_Date': datetime.datetime(2022, 2, 2, 14, 24), 'Date': '2022-02-02', 'Create_datetime_str': '2022-02-02 14:24:00', 'Geohash3': 'r1q', 'Geohash5': 'r1q56'}, {'latitude': -36.5509, 'longitude': 144.1143, 'confidence': 76, 'surface_temperature_celcius': 50, 'Producer_id': 'AQUA_Producer_02', 'Created_Date': datetime.datetime(2022, 2, 2, 19, 12), 'Date': '2022-02-02', 'Create_datetime_str': '2022-02-02 19:12:00', 'Geohash3': 'r1w', 'Geohash5': 'r1w2p'}, {'latitude': -36.7054, 'longitude': 141.6708, 'confidence': 75, 'surface_temperature_celcius': 48,

...

Climate_lst:  [{'station': 948703, 'date': '2022-02-21', 'air_temperature_celcius': 17, 'relative_humidity': 46.3, 'windspeed_knots': 11.7, 'max_wind_speed': 20.0, 'precitipation': None, 'GHI_w/m2': 150, 'geohash3': 'r1r', 'Fire_Cause': 'other', '_id': ObjectId('628a725a25967c93f524f138')}]
Hotspot_lst:  [{'latitude': -36.0914, 'longitude': 145.0299, 'confidence': 78, 'surface_temperature_celcius': 51, 'Producer_id': 'TERRA_Producer_03', 'Created_Date': datetime.datetime(2022, 2, 21, 14, 24), 'Date': '2022-02-21', 'Create_datetime_str': '2022-02-21 14:24:00', 'Geohash3': 'r1x', 'Geohash5': 'r1x4s'}, {'latitude': -34.339, 'longitude': 142.2459, 'confidence': 63, 'surface_temperature_celcius': 41, 'Producer_id': 'AQUA_Producer_02', 'Created_Date': datetime.datetime(2022, 2, 21, 19, 12), 'Date': '2022-02-21', 'Create_datetime_str': '2022-02-21 19:12:00', 'Geohash3': 'r1v', 'Geohash5': 'r1vhs'}, {'latitude': -36.8241, 'longitude': 145.1274, 'confidence': 56, 'surface_temperature_celcius': 

Climate_lst:  [{'station': 948703, 'date': '2022-02-26', 'air_temperature_celcius': 13, 'relative_humidity': 51.1, 'windspeed_knots': 6.0, 'max_wind_speed': 13.0, 'precitipation': None, 'GHI_w/m2': 110, 'geohash3': 'r1k', 'Fire_Cause': 'other', '_id': ObjectId('628a728c25967c93f524f141')}]
Hotspot_lst:  [{'latitude': -37.3863, 'longitude': 142.8822, 'confidence': 85, 'surface_temperature_celcius': 59, 'Producer_id': 'TERRA_Producer_03', 'Created_Date': datetime.datetime(2022, 2, 26, 14, 24), 'Date': '2022-02-26', 'Create_datetime_str': '2022-02-26 14:24:00', 'Geohash3': 'r1m', 'Geohash5': 'r1me7'}, {'latitude': -36.4466, 'longitude': 141.2471, 'confidence': 51, 'surface_temperature_celcius': 40, 'Producer_id': 'AQUA_Producer_02', 'Created_Date': datetime.datetime(2022, 2, 26, 19, 12), 'Date': '2022-02-26', 'Create_datetime_str': '2022-02-26 19:12:00', 'Geohash3': 'r1s', 'Geohash5': 'r1s2w'}, {'latitude': -37.6865, 'longitude': 148.5154, 'confidence': 66, 'surface_temperature_celcius': 

Climate_lst:  [{'station': 948703, 'date': '2022-03-02', 'air_temperature_celcius': 12, 'relative_humidity': 44.4, 'windspeed_knots': 7.8, 'max_wind_speed': 14.0, 'precitipation': None, 'GHI_w/m2': 108, 'geohash3': 'r1u', 'Fire_Cause': 'other', '_id': ObjectId('628a72b425967c93f524f149')}]
Hotspot_lst:  [{'latitude': -37.7168, 'longitude': 149.1369, 'confidence': 77, 'surface_temperature_celcius': 50, 'Producer_id': 'TERRA_Producer_03', 'Created_Date': datetime.datetime(2022, 3, 2, 14, 24), 'Date': '2022-03-02', 'Create_datetime_str': '2022-03-02 14:24:00', 'Geohash3': 'r36', 'Geohash5': 'r3613'}, {'latitude': -37.8519, 'longitude': 147.2555, 'confidence': 59, 'surface_temperature_celcius': 40, 'Producer_id': 'AQUA_Producer_02', 'Created_Date': datetime.datetime(2022, 3, 2, 19, 12), 'Date': '2022-03-02', 'Create_datetime_str': '2022-03-02 19:12:00', 'Geohash3': 'r32', 'Geohash5': 'r328w'}, {'latitude': -38.0427, 'longitude': 141.0271, 'confidence': 77, 'surface_temperature_celcius': 50

Climate_lst:  [{'station': 948703, 'date': '2022-03-06', 'air_temperature_celcius': 14, 'relative_humidity': 47.4, 'windspeed_knots': 7.1, 'max_wind_speed': 14.0, 'precitipation': None, 'GHI_w/m2': 123, 'geohash3': 'r1k', 'Fire_Cause': 'other', '_id': ObjectId('628a72dc25967c93f524f150')}]
Hotspot_lst:  [{'latitude': -35.2378, 'longitude': 142.9864, 'confidence': 100, 'surface_temperature_celcius': 94, 'Producer_id': 'TERRA_Producer_03', 'Created_Date': datetime.datetime(2022, 3, 6, 14, 24), 'Date': '2022-03-06', 'Create_datetime_str': '2022-03-06 14:24:00', 'Geohash3': 'r1t', 'Geohash5': 'r1txt'}, {'latitude': -35.8125, 'longitude': 142.1286, 'confidence': 75, 'surface_temperature_celcius': 109, 'Producer_id': 'AQUA_Producer_02', 'Created_Date': datetime.datetime(2022, 3, 6, 19, 12), 'Date': '2022-03-06', 'Create_datetime_str': '2022-03-06 19:12:00', 'Geohash3': 'r1t', 'Geohash5': 'r1th6'}, {'latitude': -37.335, 'longitude': 148.064, 'confidence': 88, 'surface_temperature_celcius': 60

Climate_lst:  [{'station': 948703, 'date': '2022-03-12', 'air_temperature_celcius': 22, 'relative_humidity': 55.5, 'windspeed_knots': 10.3, 'max_wind_speed': 19.0, 'precitipation': None, 'GHI_w/m2': 180, 'geohash3': 'r1s', 'Fire_Cause': 'other', '_id': ObjectId('628a732225967c93f524f159')}]
Hotspot_lst:  [{'latitude': -36.6228, 'longitude': 142.428, 'confidence': 67, 'surface_temperature_celcius': 43, 'Producer_id': 'TERRA_Producer_03', 'Created_Date': datetime.datetime(2022, 3, 12, 14, 24), 'Date': '2022-03-12', 'Create_datetime_str': '2022-03-12 14:24:00', 'Geohash3': 'r1m', 'Geohash5': 'r1mr9'}, {'latitude': -36.7611, 'longitude': 141.6048, 'confidence': 68, 'surface_temperature_celcius': 44, 'Producer_id': 'AQUA_Producer_02', 'Created_Date': datetime.datetime(2022, 3, 12, 19, 12), 'Date': '2022-03-12', 'Create_datetime_str': '2022-03-12 19:12:00', 'Geohash3': 'r1k', 'Geohash5': 'r1kwy'}, {'latitude': -36.2377, 'longitude': 141.1427, 'confidence': 73, 'surface_temperature_celcius': 